In [1]:
import pandas as pd
import numpy as np
import sktime
import seaborn as sns
import matplotlib.pyplot as plt
from convertcsv.import_preprocess_v4 import readcsvs
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sktime.transformations.panel.rocket import Rocket
from sktime.transformations.panel.rocket import MiniRocketMultivariate
from sktime.classification.kernel_based import RocketClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.impute import KNNImputer
from convertcsv.get_all_metrics_with_tags import get_all_metrics_with_tags
from visualization import graphs
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sktime.datatypes import convert_to
from sktime.datatypes import MTYPE_REGISTER



In [2]:
#What should we do here?

#Set up many different pipelines to compare.

#The things to vary, in order of significance/importance
"""
NaN treshold
"""
file_list, y = get_all_metrics_with_tags(r"F:\Master\Kubernetes\sockshop\microservices-demo\query\automated\generated_csvs_4")
initial_readings = readcsvs(file_list, reduce_NaNs_treshold=False, remove_unique_cols=True, remove_monotonic_increasing=True)
initial_readings



go_gc_duration_seconds&payment:80&payment&0.25  \
instances timepoints                                                            
0         2023-03-24 14:09:48                                        0.000307   
          2023-03-24 14:09:53                                        0.000307   
          2023-03-24 14:09:58                                        0.000307   
          2023-03-24 14:10:03                                        0.000307   
          2023-03-24 14:10:08                                        0.000307   
...                                                                       ...   
75        2023-03-29 16:21:34                                        0.000318   
          2023-03-29 16:21:39                                        0.000318   
          2023-03-29 16:21:44                                        0.000318   
          2023-03-29 16:21:49                                        0.000318   
          2023-03-29 16:21:54                                        0.000318   

                               go_gc_duration_seconds&payment:80&payment&0.75  \
instances timepoints                                                            
0         2023-03-24 14:09:48                                        0.000333   
          2023-03-24 14:09:53                                        0.000333   
          2023-03-24 14:09:58                                        0.000333   
          2023-03-24 14:10:03                                        0.000333   
          2023-03-24 14:10:08                                        0.000333   
...                                                                       ...   
75        2023-03-29 16:21:34                                        0.000370   
          2023-03-29 16:21:39                                        0.000370   
          2023-03-29 16:21:44                                        0.000370   
          2023-03-29 16:21:49                                        0.000370   
          2023-03-29 16:21:54                                        0.000370   

                               go_gc_duration_seconds&user:80&user&0.25  \
instances timepoints                                                      
0         2023-03-24 14:09:48                                  0.000321   
          2023-03-24 14:09:53                                  0.000321   
          2023-03-24 14:09:58                                  0.000321   
          2023-03-24 14:10:03                                  0.000321   
          2023-03-24 14:10:08                                  0.000321   
...                                                                 ...   
75        2023-03-29 16:21:34                                  0.000380   
          2023-03-29 16:21:39                                  0.000380   
          2023-03-29 16:21:44                                  0.000380   
          2023-03-29 16:21:49                                  0.000380   
          2023-03-29 16:21:54                                  0.000380   

                               go_gc_duration_seconds&user:80&user&0.5  \
instances timepoints                                                     
0         2023-03-24 14:09:48                                 0.000333   
          2023-03-24 14:09:53                                 0.000333   
          2023-03-24 14:09:58                                 0.000333   
          2023-03-24 14:10:03                                 0.000333   
          2023-03-24 14:10:08                                 0.000333   
...                                                                ...   
75        2023-03-29 16:21:34                                 0.000451   
          2023-03-29 16:21:39                                 0.000451   
          2023-03-29 16:21:44                                 0.000451   
          2023-03-29 16:21:49                                 0.000451   
          2023-03-29 16:21:54                              

In [3]:
#Prior testing revealed optimal KNN imputation
imputer = KNNImputer(n_neighbors=1)
complete_value_set = imputer.fit_transform(initial_readings)
imputed_df = pd.DataFrame(complete_value_set, index=initial_readings.index, columns=initial_readings.columns)
imputed_df.index.get_level_values(1)

DatetimeIndex(['2023-03-24 14:09:48', '2023-03-24 14:09:53',
               '2023-03-24 14:09:58', '2023-03-24 14:10:03',
               '2023-03-24 14:10:08', '2023-03-24 14:10:13',
               '2023-03-24 14:10:18', '2023-03-24 14:10:23',
               '2023-03-24 14:10:28', '2023-03-24 14:10:33',
               ...
               '2023-03-29 16:21:09', '2023-03-29 16:21:14',
               '2023-03-29 16:21:19', '2023-03-29 16:21:24',
               '2023-03-29 16:21:29', '2023-03-29 16:21:34',
               '2023-03-29 16:21:39', '2023-03-29 16:21:44',
               '2023-03-29 16:21:49', '2023-03-29 16:21:54'],
              dtype='datetime64[ns]', name='timepoints', length=10073, freq=None)

In [4]:
#split into training x and y
X_train, X_test, y_train, y_test = train_test_split(convert_to(imputed_df,to_type="df-list"), y)
print(MTYPE_REGISTER)
#X_train = convert_to(X_train, to_type="pd-multiindex")


[('pd.Series', 'Series', 'pd.Series representation of a univariate series'), ('pd.DataFrame', 'Series', 'pd.DataFrame representation of a uni- or multivariate series'), ('np.ndarray', 'Series', '2D numpy.ndarray with rows=samples, cols=variables, index=integers'), ('xr.DataArray', 'Series', 'xr.DataArray representation of a uni- or multivariate series'), ('dask_series', 'Series', 'xdas representation of a uni- or multivariate series'), ('nested_univ', 'Panel', 'pd.DataFrame with one column per variable, pd.Series in cells'), ('numpy3D', 'Panel', '3D np.array of format (n_instances, n_columns, n_timepoints)'), ('numpyflat', 'Panel', 'WARNING: only for internal use, not a fully supported Panel mtype. 2D np.array of format (n_instances, n_columns*n_timepoints)'), ('pd-multiindex', 'Panel', 'pd.DataFrame with multi-index (instances, timepoints)'), ('pd-wide', 'Panel', 'pd.DataFrame in wide format, cols = (instance*timepoints)'), ('pd-long', 'Panel', 'pd.DataFrame in long format, cols = (in

In [5]:
from sktime.registry import all_estimators
all_estimators("classifier",filter_tags={"capability:unequal_length":True})

f:\anaconda\envs\py39\lib\site-packages\sktime\utils\validation\_dependencies.py:143: UserWarning: str requires package 'dtw-python' to be present in the python environment, but 'dtw-python' was not found. 'dtw-python' is a soft dependency and not included in the base sktime installation. Please run: `pip install dtw-python` to install the dtw-python package. To install all soft dependencies, run: `pip install sktime[all_extras]`
  warnings.warn(msg)
f:\anaconda\envs\py39\lib\site-packages\sktime\utils\validation\_dependencies.py:143: UserWarning: No module named 'pyod'. 'pyod' is a soft dependency and not included in the base sktime installation. Please run: `pip install pyod` to install the pyod package. To install all soft dependencies, run: `pip install sktime[all_extras]`
  warnings.warn(msg)
f:\anaconda\envs\py39\lib\site-packages\sktime\utils\validation\_dependencies.py:143: UserWarning: No module named 'hmmlearn'. 'hmmlearn.hmm' is a soft dependency and not included in the base

[('DummyClassifier', sktime.classification.dummy._dummy.DummyClassifier),
 ('KNeighborsTimeSeriesClassifier',
  sktime.classification.distance_based._time_series_neighbors.KNeighborsTimeSeriesClassifier),
 ('TimeSeriesSVC', sktime.classification.kernel_based._svc.TimeSeriesSVC)]

Padding/reduction

In [11]:
#Find the length of each individual time series
def padding(df:pd.DataFrame):
    instances = df.index.get_level_values(0).unique()
    for instance in instances:
        #This line of code gets every second level index value that corresponds to the current first level index value. 
        second_level_values = df.index.get_level_values(1)[df.index.get_level_values(0)==instance]
        length = len(second_level_values)
        print( "{} length of values is: {}".format(instance, length))


padding(imputed_df)


In [15]:
from sktime.datatypes import check_raise

X_train.index


<function list.index(value, start=0, stop=9223372036854775807, /)>

In [17]:
"hi"

'hi'

In [8]:
rocketclass = RocketClassifier()

rocketclass.fit(X=X_train, y=np.array(y_train))
rocketclass.score(X_test, np.array(y_test))


ValueError: Data seen by RocketClassifier instance has unequal length series, but this RocketClassifier instance cannot handle unequal length series. Calls with unequal length series may result in error or unreliable results.